In [3]:
import sys
import os
import pandas as pd
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..','..','..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from argparse import Namespace
from constants.paths import FOLDER_PATH
from examples.benchmark import local_get_args
from load_inputs.bike_in import load_data, START, END
from plotting.TS_analysis import plot_TS



coverage_period = pd.date_range(start=START, end=END, freq='15min')[:-1]  # Exclude the last date to match the data
invalid_dates = []
minmaxnorm = True
standardize = False  # Set to True if you want to standardize the data
config = {'model_name': 'STGCN',
         'freq' : '15min',
        'dataset_names':['bike_in'],
        'dataset_for_coverage' : ['bike_in'],
        'target_data': 'bike_in',
        'contextual_kwargs':{'bike_in':{'agg_iris_target_n':50,
                                        'threshold_volume_min': 0,
                                },
                                        },
        'step_ahead':1,
        'horizon_step':1,
        }

args = local_get_args(config['model_name'],
                args_init = None,
                dataset_names=config['dataset_names'],
                dataset_for_coverage=config['dataset_for_coverage'],
                modification = config)


ds = load_data(FOLDER_PATH, coverage_period, invalid_dates, args, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)

df =  pd.DataFrame(ds.U_train[:,:,-1].detach().cpu().numpy(),columns = ds.spatial_unit, index = ds.tensor_limits_keeper.df_verif_train.iloc[:,-2] )
plot_TS(df.iloc[:2000,:],bool_show = True, height = 800)

Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_attracted_by_station15min.csv...
df pivoted:  (70094, 435)
df reindexed :  (35040, 435)
Len coverage period:  35040
df filtered:  (35040, 435)
   Loaded data: (35040, 435)
   Dimension after spatial agg: (35040, 27)
Values with issues:  40.741%
Regular Values that we have to set to 0:  0.000%
Values with issues:  40.741%
Regular Values that we have to set to 0:  0.818%
Values with issues:  40.741%
Regular Values that we have to set to 0:  1.201%
Values with issues:  40.741%
Regular Values that we have to set to 0:  0.000%
Values with issues:  40.741%
Regular Values that we have to set to 0:  0.822%
Values with issues:  40.741%
Regular Values that we have to set to 0:  1.199%


Loading BokehJS ...

figure(id='p1206', ...)

In [ ]:
from load_inputs.bike_in import FILE_PATTERN,DATA_SUBFOLDER,LOCATION_COL,VALUE_COL,DATE_COL

file_pattern= FILE_PATTERN
data_subfolder = DATA_SUBFOLDER
location_col = LOCATION_COL
value_col = VALUE_COL
date_col = DATE_COL


target_freq = args.freq
# Construction spécifique du nom de fichier pour velov
file_name = f"{file_pattern}{target_freq}"
data_file_path = os.path.join(FOLDER_PATH, data_subfolder, f"{file_name}.csv")

print(f"Loading from {data_file_path}...")
try:
    df = pd.read_csv(data_file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"ERROR : file {data_file_path} does not exists.\nPlease check path and  frequency '{target_freq}', it has to exists for velov_{direction}")
except Exception as e:
    raise ImportError(f"ERROR while loading {file_name}.csv: {e}")

# --- Preprocessing ---
df[date_col] = pd.to_datetime(df[date_col])
df_pivoted = df.pivot_table(index=date_col, columns=location_col, values=value_col, aggfunc='sum')

# Fill Nan value
df_pivoted = df_pivoted.fillna(0)

# Convert into Datetime
df_pivoted.index = pd.to_datetime(df_pivoted.index)
print('df pivoted: ',df_pivoted.shape)
df_pivoted = df_pivoted.reindex(pd.date_range(start =START, end = END, freq=target_freq)[:-1]).fillna(0)
print('df reindexed : ',df_pivoted.shape)
print('Len coverage period: ',len(coverage_period))
# Temporal filtering on 'coverage_period'
df_filtered = df_pivoted[df_pivoted.index.isin(coverage_period)].copy()
print('df filtered: ',df_filtered.shape)

if df_filtered.empty:
        raise ImportError(f"ERRROR : not any remaining data after temporal filtering on {file_name}.csv.\nPlease check dataset_coverage: {args.dataset_coverage} and dataset_names {args.dataset_names}")
print(f"   Loaded data: {df_filtered.shape}")



# Spatial Agg: 
if ('agg_iris_target_n' in args.contextual_kwargs[name].keys()) and (args.contextual_kwargs[name]['agg_iris_target_n'] is not None):
    target_n = args.contextual_kwargs[name]['agg_iris_target_n']
    threshold_volume_min = args.contextual_kwargs[name]['threshold_volume_min']

    #Load Data: 
    s_zone2stations_path = f"{FOLDER_PATH}/lyon_iris_agg{target_n}/zone2stations.csv"
    s_zone2stations = pd.read_csv(s_zone2stations_path,index_col = 0)

    agg_df = pd.DataFrame(columns = s_zone2stations.index)
    for idx,row in s_zone2stations.iterrows():
        station_id = row.STATION
        columns = list(map(int,station_id.split(' ')))
        effective_columns = [c for c in columns if c in df_filtered.columns]
        agg_df[idx] = df_filtered[effective_columns].sum(axis=1)

    mask = agg_df.mean() > threshold_volume_min
    df_filtered = agg_df.T[mask].T
    print(f"   Dimension after spatial agg: {df_filtered.shape}")


In [4]:
from load_inputs.bike_out import load_data as load_data_out

args_out = Namespace(**vars(args))  
args_out.target_data = 'bike_out'
args_out.dataset_for_coverage = ['bike_out']
args_out.dataset_names = ['bike_out']
args_out.contextual_kwargs = {'bike_out':{'agg_iris_target_n':50,
                                          'threshold_volume_min': 1,
                                         
                                        },
                            }


ds_out = load_data_out(FOLDER_PATH, coverage_period, invalid_dates, args_out, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)
df_out =  pd.DataFrame(ds_out.U_train[:,:,-1].detach().cpu().numpy(),columns = ds_out.spatial_unit, index = ds_out.tensor_limits_keeper.df_verif_train.iloc[:,-2] )


# Filtered
args_out.contextual_kwargs = {'bike_out':{'agg_iris_target_n':50,
                                          'threshold_volume_min': 1,
                                           'quantile_filter_outliers': 0.98
                                        },
                            }
ds_out_filtered = load_data_out(FOLDER_PATH, coverage_period, invalid_dates, args_out, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)
df_out_filtered =  pd.DataFrame(ds_out_filtered.U_train[:,:,-1].detach().cpu().numpy(),columns = [f"filtered_{c}" for c in ds_out_filtered.spatial_unit], index = ds_out_filtered.tensor_limits_keeper.df_verif_train.iloc[:,-2] )
# ---

df_out_concat = pd.concat([df_out, df_out_filtered], axis=1)
plot_TS(df_out_concat.iloc[:2000,:],bool_show = True, height = 800)

Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...


df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 13)
Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 13)


Loading BokehJS ...

figure(id='p1411', ...)

In [6]:
import numpy as np 
df_all = df_out.sum(1)
df_all[df_all.index.hour.isin([0,1,2,3,4,5,6])] = df_all[df_all.index.hour.isin([0,1,2,3,4,5,6])] +1
plot_TS(pd.DataFrame(df_all),bool_show = True, height = 800,title = 'Detection of Missing values')

Loading BokehJS ...

figure(id='p1772', ...)